# アノテーションした結果の統計情報を可視化します。
2023/12/12時点の結果になります。  
`crassone-analysis/data/annotation/`にlabelstudioからexportしたjsonファイルを置いてください。

In [1]:
import glob
import json
import os
import pandas as pd

In [2]:
pattern_date = '2023-12-12'

In [3]:
anno_info_dir = f'{"/".join(os.getcwd().split("/")[:-2])}/data/annotation'
anno_info_path = [path for path in glob.glob(f'{anno_info_dir}/*.json') if pattern_date in path][0]
anno_dict = json.load(open(anno_info_path))

In [4]:
anno_dict_not_cancelled = [anno for anno in anno_dict if not anno['annotations'][0]['was_cancelled']]

In [5]:
labels_list = [
    "helmet",
    "unsafe-helmet (no chin strap)",
    "unsafe-helmet (inadequate covering)",
    "no-helmet",
    "helmet unknown (small)",
    "helmet unknown (occlusion)",
    "safety belt",
    "person",
    "person in high place",
]

In [6]:
labels_count_by_image = {key: 0 for key in labels_list}
for anno_by_img in anno_dict_not_cancelled:
    label_bool_by_img = {key: False for key in labels_list}
    for annotation in anno_by_img['annotations']:
        for result in annotation['result']:
            labels = result['value']['rectanglelabels']
            for label in labels:
                label_bool_by_img[label] = True
    for key, value in label_bool_by_img.items():
        labels_count_by_image[key] += int(value)

In [7]:
labels_count_by_bbox = {key: 0 for key in labels_list}
for anno_by_img in anno_dict_not_cancelled:
    for annotation in anno_by_img['annotations']:
        for result in annotation['result']:
            labels = result['value']['rectanglelabels']
            for label in labels:
                labels_count_by_bbox[label] += 1

In [8]:
labels_count_by_image_in_fixed_camera = {key: 0 for key in labels_list}
for anno_by_img in anno_dict_not_cancelled:
    if not 'fixed-point-camera' in anno_by_img['data']['image']:
        continue
    label_bool_by_img = {key: False for key in labels_list}
    for annotation in anno_by_img['annotations']:
        for result in annotation['result']:
            labels = result['value']['rectanglelabels']
            for label in labels:
                label_bool_by_img[label] = True
    for key, value in label_bool_by_img.items():
        labels_count_by_image_in_fixed_camera[key] += int(value)

In [9]:
labels_count_by_image_in_learning = {key: 0 for key in labels_list}
for anno_by_img in anno_dict_not_cancelled:
    if not 'for-learning' in anno_by_img['data']['image']:
        continue
    label_bool_by_img = {key: False for key in labels_list}
    for annotation in anno_by_img['annotations']:
        for result in annotation['result']:
            labels = result['value']['rectanglelabels']
            for label in labels:
                label_bool_by_img[label] = True
    for key, value in label_bool_by_img.items():
        labels_count_by_image_in_learning[key] += int(value)

In [10]:
labels_count_by_image_in_indoor = {key: 0 for key in labels_list}
for anno_by_img in anno_dict_not_cancelled:
    if not 'fixed-point-camera' in anno_by_img['data']['image']:
        continue
    if not 'indoor' in anno_by_img['data']['image']:
        continue
    label_bool_by_img = {key: False for key in labels_list}
    for annotation in anno_by_img['annotations']:
        for result in annotation['result']:
            labels = result['value']['rectanglelabels']
            for label in labels:
                label_bool_by_img[label] = True
    for key, value in label_bool_by_img.items():
        labels_count_by_image_in_indoor[key] += int(value)

In [11]:
labels_count_by_image_in_outdoor = {key: 0 for key in labels_list}
for anno_by_img in anno_dict_not_cancelled:
    if not 'fixed-point-camera' in anno_by_img['data']['image']:
        continue
    if not 'outdoor' in anno_by_img['data']['image']:
        continue
    label_bool_by_img = {key: False for key in labels_list}
    for annotation in anno_by_img['annotations']:
        for result in annotation['result']:
            labels = result['value']['rectanglelabels']
            for label in labels:
                label_bool_by_img[label] = True
    for key, value in label_bool_by_img.items():
        labels_count_by_image_in_outdoor[key] += int(value)

In [12]:
labels_count_by_image_in_learning_indoor = {key: 0 for key in labels_list}
for anno_by_img in anno_dict_not_cancelled:
    if not 'for-learning' in anno_by_img['data']['image']:
        continue
    if (
        '2023-11-19-omaezaki-500' in anno_by_img['data']['image'] or 
        '2023-11-23-mie-safetybelt' in anno_by_img['data']['image']
    ):
        continue
    label_bool_by_img = {key: False for key in labels_list}
    for annotation in anno_by_img['annotations']:
        for result in annotation['result']:
            labels = result['value']['rectanglelabels']
            for label in labels:
                label_bool_by_img[label] = True
    for key, value in label_bool_by_img.items():
        labels_count_by_image_in_learning_indoor[key] += int(value)

In [13]:
labels_count_by_image_in_learning_outdoor = {key: 0 for key in labels_list}
for anno_by_img in anno_dict_not_cancelled:
    if not 'for-learning' in anno_by_img['data']['image']:
        continue
    if not (
        '2023-11-19-omaezaki-500' in anno_by_img['data']['image'] or 
        '2023-11-23-mie-safetybelt' in anno_by_img['data']['image']
    ):
        continue
    label_bool_by_img = {key: False for key in labels_list}
    for annotation in anno_by_img['annotations']:
        for result in annotation['result']:
            labels = result['value']['rectanglelabels']
            for label in labels:
                label_bool_by_img[label] = True
    for key, value in label_bool_by_img.items():
        labels_count_by_image_in_learning_outdoor[key] += int(value)

In [14]:
# 全画像枚数の集計
anno_count_df = pd.DataFrame(labels_count_by_image, index=["画像枚数"]).T
anno_count_df.loc["アノテーション合計"] = len(anno_dict_not_cancelled)
anno_count_df.loc["アノテーション合計(キャンセル含む)"] = len(anno_dict)

# 全画像のbboxの集計
anno_count_bbox_df = pd.DataFrame(labels_count_by_bbox, index=["バウンディングボックス数"]).T
anno_count_bbox_df.loc["アノテーション合計"] = anno_count_bbox_df["バウンディングボックス数"].sum()

# 固定カメラの画像枚数の集計
anno_count_fixed_camera_df = pd.DataFrame(labels_count_by_image_in_fixed_camera, index=["画像枚数-固定カメラ"]).T
anno_count_fixed_camera_df.loc["アノテーション合計"] = len(
    [anno for anno in anno_dict_not_cancelled if 'fixed-point-camera' in anno['data']['image']]
)
anno_count_fixed_camera_df.loc["アノテーション合計(キャンセル含む)"] = len(
    [anno for anno in anno_dict if 'fixed-point-camera' in anno['data']['image']]
)

# 学習用の画像枚数の集計
anno_count_learning_df = pd.DataFrame(labels_count_by_image_in_learning, index=["画像枚数-学習用"]).T
anno_count_learning_df.loc["アノテーション合計"] = len(
    [anno for anno in anno_dict_not_cancelled if 'for-learning' in anno['data']['image']]
)
anno_count_learning_df.loc["アノテーション合計(キャンセル含む)"] = len(
    [anno for anno in anno_dict if 'for-learning' in anno['data']['image']]
)

# 屋内固定カメラの画像枚数の集計
anno_count_indoor_df = pd.DataFrame(labels_count_by_image_in_indoor, index=["画像枚数-固定カメラ-屋内"]).T
anno_count_indoor_df.loc["アノテーション合計"] = len(
    [
        anno for anno in anno_dict_not_cancelled 
        if 'fixed-point-camera' in anno['data']['image'] and 'indoor' in anno['data']['image']
    ]
)
anno_count_indoor_df.loc["アノテーション合計(キャンセル含む)"] = len(
    [
        anno for anno in anno_dict 
        if 'fixed-point-camera' in anno['data']['image'] and 'indoor' in anno['data']['image']
    ]
)

# 屋外固定カメラの画像枚数の集計
anno_count_outdoor_df = pd.DataFrame(labels_count_by_image_in_outdoor, index=["画像枚数-固定カメラ-屋外"]).T
anno_count_outdoor_df.loc["アノテーション合計"] = len(
    [
        anno for anno in anno_dict_not_cancelled 
        if 'fixed-point-camera' in anno['data']['image'] and 'outdoor' in anno['data']['image']
    ]
)
anno_count_outdoor_df.loc["アノテーション合計(キャンセル含む)"] = len(
    [
        anno for anno in anno_dict 
        if 'fixed-point-camera' in anno['data']['image'] and 'outdoor' in anno['data']['image']
    ]
)

# 屋内学習用の画像枚数の集計
anno_count_learning_indoor_df = pd.DataFrame(labels_count_by_image_in_learning_indoor, index=["画像枚数-学習用-屋内"]).T
anno_count_learning_indoor_df.loc["アノテーション合計"] = len(
    [
        anno for anno in anno_dict_not_cancelled 
        if 'for-learning' in anno['data']['image'] 
        and '2023-11-19-omaezaki-500' not in anno['data']['image'] 
        and '2023-11-23-mie-safetybelt' not in anno['data']['image']
    ]
)
anno_count_learning_indoor_df.loc["アノテーション合計(キャンセル含む)"] = len(
    [
        anno for anno in anno_dict 
        if 'for-learning' in anno['data']['image'] 
        and '2023-11-19-omaezaki-500' not in anno['data']['image'] 
        and '2023-11-23-mie-safetybelt' not in anno['data']['image']
    ]
)

# 屋外学習用の画像枚数の集計
anno_count_learning_outdoor_df = pd.DataFrame(labels_count_by_image_in_learning_outdoor, index=["画像枚数-学習用-屋外"]).T
anno_count_learning_outdoor_df.loc["アノテーション合計"] = len(
    [
        anno for anno in anno_dict_not_cancelled 
        if 'for-learning' in anno['data']['image'] 
        and (
            '2023-11-19-omaezaki-500' in anno['data']['image'] or 
            '2023-11-23-mie-safetybelt' in anno['data']['image']
        )
    ]
)
anno_count_learning_outdoor_df.loc["アノテーション合計(キャンセル含む)"] = len(
    [
        anno for anno in anno_dict 
        if 'for-learning' in anno['data']['image'] 
        and (
            '2023-11-19-omaezaki-500' in anno['data']['image'] or 
            '2023-11-23-mie-safetybelt' in anno['data']['image']
        )
    ]
)

In [15]:
anno_count_whole_df = pd.concat([
    anno_count_df,
    anno_count_bbox_df,
    anno_count_fixed_camera_df,
    anno_count_learning_df,
], axis=1)
anno_count_whole_df

,画像枚数,バウンディングボックス数,画像枚数-固定カメラ,画像枚数-学習用
helmet,846,850.0,38,808
unsafe-helmet (no chin strap),415,415.0,34,381
unsafe-helmet (inadequate covering),534,629.0,365,169
no-helmet,385,514.0,128,257
helmet unknown (small),197,205.0,108,89
helmet unknown (occlusion),691,746.0,252,439
safety belt,1427,1430.0,0,1427
person,2600,3303.0,590,2010
person in high place,110,115.0,80,30
アノテーション合計,2697,8207.0,647,2050


In [16]:
anno_count_whole_for_learning_df = pd.concat([
    anno_count_df,
    anno_count_bbox_df,
    anno_count_fixed_camera_df,
    anno_count_learning_outdoor_df,
    anno_count_learning_indoor_df,
], axis=1)
anno_count_whole_for_learning_df

,画像枚数,バウンディングボックス数,画像枚数-固定カメラ,画像枚数-学習用-屋外,画像枚数-学習用-屋内
helmet,846,850.0,38,169,639
unsafe-helmet (no chin strap),415,415.0,34,275,106
unsafe-helmet (inadequate covering),534,629.0,365,168,1
no-helmet,385,514.0,128,47,210
helmet unknown (small),197,205.0,108,36,53
helmet unknown (occlusion),691,746.0,252,111,328
safety belt,1427,1430.0,0,366,1061
person,2600,3303.0,590,752,1258
person in high place,110,115.0,80,30,0
アノテーション合計,2697,8207.0,647,782,1268


In [17]:
anno_count_fixed_camera_df = pd.concat([
    anno_count_df,
    anno_count_fixed_camera_df,
    anno_count_indoor_df,
    anno_count_outdoor_df,
], axis=1)
anno_count_fixed_camera_df

,画像枚数,画像枚数-固定カメラ,画像枚数-固定カメラ-屋内,画像枚数-固定カメラ-屋外
helmet,846,38,5,33
unsafe-helmet (no chin strap),415,34,6,28
unsafe-helmet (inadequate covering),534,365,125,240
no-helmet,385,128,8,120
helmet unknown (small),197,108,13,95
helmet unknown (occlusion),691,252,89,163
safety belt,1427,0,0,0
person,2600,590,148,442
person in high place,110,80,0,80
アノテーション合計,2697,647,148,499


In [18]:
anno_count_learning_df = pd.concat([
    anno_count_df,
    anno_count_learning_df,
    anno_count_learning_indoor_df,
    anno_count_learning_outdoor_df,
], axis=1)
anno_count_learning_df

,画像枚数,画像枚数-学習用,画像枚数-学習用-屋内,画像枚数-学習用-屋外
helmet,846,808,639,169
unsafe-helmet (no chin strap),415,381,106,275
unsafe-helmet (inadequate covering),534,169,1,168
no-helmet,385,257,210,47
helmet unknown (small),197,89,53,36
helmet unknown (occlusion),691,439,328,111
safety belt,1427,1427,1061,366
person,2600,2010,1258,752
person in high place,110,30,0,30
アノテーション合計,2697,2050,1268,782
